In [1]:
# Install vector database
# ! pip install vectordb
# !pip install faiss-cpu
# !pip install faiss-gpu
# CONDA
# !conda install -c conda-forge faiss-gpu
# !conda install -c conda-forge transformers

## Import modules


In [1]:
from utils import clean_dbs, get_all_feats, get_all_docs, reformat_keyframe, visualize, filter_by_detail_scripts
from vector_database import TextEmbedding, FrameDoc, VectorDB
from const import *
from filter_by_audio import predict, filter_by_audio
# 

Get all script...
Finished!
Initalize Tokenizer and BERT model


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Finished!
Set up Bert and faiss
Finished!


In [2]:
clean_dbs()
reformat_keyframe()

### Get all features files


In [3]:
all_feat_files = get_all_feats()
# print(all_feat_files)
print(len(all_feat_files)) 

737


### Create all the Docs


In [4]:
doc_list = get_all_docs(all_feat_files)

In [5]:
print(len(doc_list))
print(doc_list[100])

202148

            Video name: L01_V001
            Image path: ../data/keyframes/L01_V001/0101.jpg
            Keyframe Id: 101
            Actual keyframe idx: 11047
            Time: 441.88
            FPS: 25
            Metadata: {'publish_date': '01/12/2022', 'watch_url': 'https://youtube.com/watch?v=HNsRpkryGXA'}
          


## DEMO


### Create DB


In [6]:
DB = VectorDB()
DB.index(doc_list)

### Query


### Filter 1


In [7]:
results1 = DB.search(
    "Scene of a car driving onto the sidewalk and hitting 4 motorbikes. There was a person sitting nearby buying things from a stall on the roadside. It ends with a scene of someone running out of the house to see the situation.",
    1000,
)  # Nên lấy nhiều

In [9]:
visualize(filter_by_audio(results1[:1000], [""]), True)

In [ ]:
predict(["tai nạn", "lề đường"], results1, "query-p2-28.csv", mode = 1)

In [10]:
visualize(filter_by_detail_scripts("mô phỏng"), False)